<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


# Image Classification Competition Model Submission Guide

---

Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data / Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [2]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/neuralnet_competition_data-repository:latest') 


Data downloaded successfully.


In [ ]:
# Create training data objects 

# Extract images
!unzip "neuralnet_competition_data/X_train.zip"
!unzip "neuralnet_competition_data/X_test.zip" 

# Create ordered list of filepaths 
train_filepaths = [('/content/train_shuffle/' + str(i) + '.jpg') for i in range(0, 6472)]
test_filepaths = [('/content/test_shuffle/' + str(i) + '.jpg') for i in range(0, 9127)]

# Read in y_train data 
import pandas as pd 
y_train = pd.read_csv("neuralnet_competition_data/y_train.csv")

##2.   Preprocess data / Write and Save Preprocessor function


In [4]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(image_filepath, shape=(10, 10)):
        """
        This function preprocesses reads in images, resizes them to a fixed shape and
        min/max transforms them before converting feature values to float32 numeric values
        required by onnx files.
        
        params:
            image_filepath
                full filepath of a particular image
                      
        returns:
            X
                numpy array of preprocessed image data
                  
        """
           
        import cv2
        import numpy as np

        "Resize a color image and min/max transform the image"
        img = cv2.imread(image_filepath) # Read in image from filepath.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of blue green and red, we reverse the order for ML.
        img = cv2.resize(img, shape) # Change height and width of image.
        img = img / 255.0 # Min-max transform.


        # Resize all the images...
        X = np.array(img)
        X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels] for keras model.
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        return X

In [5]:
# Use preprocessor to create X_train object 

# Import image, load to array of shape height, width, channels, then min/max transform...
# Read in all images from filenames...
import numpy as np 
import os 

preprocessed_image_data = [preprocessor(x) for x in train_filepaths]

# Object needs to be an array rather than a list for Keras. (vstack converts above list to array object.)
X_train = np.vstack(preprocessed_image_data)
# Assigning to X_train to highlight that this represents feature input data for our model.

In [6]:
# Preprocess X_test image data to generate predictions from models 
import numpy as np

preprocessed_image_data = [preprocessor(x) for x in test_filepaths]

#Create single X_test array from preprocessed images
X_test = np.vstack(preprocessed_image_data) 

##3. Fit model on preprocessed data and save preprocessor function and model 


In [7]:
# Let's build a convnet model...
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers

with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
                                 
        cnn1 = Sequential()
        cnn1.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=[10, 10, 3]))
        cnn1.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
        cnn1.add(MaxPooling2D(pool_size=2)) 
        cnn1.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
        cnn1.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
        cnn1.add(MaxPooling2D(pool_size=2))
        cnn1.add(Dropout(0.3))
        cnn1.add(Flatten())
        cnn1.add(Dense(50, activation='relu'))
        cnn1.add(Dropout(0.2))
        cnn1.add(Dense(3, activation='softmax'))

        # Compile model...
        cnn1.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

        # Fitting the NN to the Training set...
        hist = cnn1.fit(X_train, y_train, epochs=2, verbose=1, validation_split=0.2)

Epoch 1/2
162/162 [==============================] - 4s 15ms/step - loss: 1.1785 - accuracy: 0.4425 - val_loss: 1.0904 - val_accuracy: 0.4734
Epoch 2/2
162/162 [==============================] - 2s 13ms/step - loss: 1.0019 - accuracy: 0.5548 - val_loss: 0.8505 - val_accuracy: 0.6587


#### Save preprocessor function to local "preprocessor.zip" file

In [8]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [9]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(cnn1, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [10]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials

# Note -- This is the unique rest api that powers this specific image classification Model Plaground
# ... Update the apiurl if submitting to a new competition

apiurl="https://ez2ijee9v3.execute-api.us-east-1.amazonaws.com/prod/m"
set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [11]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [12]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=cnn1.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

286/286 [==============================] - 1s 4ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 1

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2637


In [13]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dropout_layers,flatten_layers,maxpooling2d_layers,conv2d_layers,dense_layers,relu_act,softmax_act,loss,optimizer,memory_size,username,version
0,52.63%,52.76%,55.94%,58.26%,keras,False,True,Sequential,11,10811,2,1,2,4,2,5,1,str,RMSprop,44936,COMS_NNDL,1


## 5. Repeat submission process to improve place on leaderboard


In [14]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).

with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
      cnn2 = Sequential()
      cnn2.add(Dense(64,  input_shape=(10, 10, 3), activation='relu'))
      cnn2.add(Dense(64, activation='relu'))
      cnn2.add(Dense(64, activation='relu'))
      cnn2.add(Flatten())
      cnn2.add(Dense(3, activation='softmax')) 
                                                  
      # Compile model
      cnn2.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

      # Fitting the NN to the Training set
      cnn2.fit(X_train, y_train, 
                    epochs = 1, verbose=1, validation_split=.2)

162/162 [==============================] - 3s 15ms/step - loss: 0.9834 - accuracy: 0.5161 - val_loss: 0.8720 - val_accuracy: 0.5961


In [15]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(cnn2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [16]:
#Submit Model 2: 

#-- Generate predicted y values (Model 2)
prediction_column_index=cnn2.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

286/286 [==============================] - 3s 8ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 2

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2637


In [17]:
# Compare two or more models
data=mycompetition.compare_models([1,2], verbose=1)
mycompetition.stylize_compare(data)

,Model_1_Layer,Model_1_Shape,Model_1_Params,Model_2_Layer,Model_2_Shape,Model_2_Params
0,Conv2D,"[None, 10, 10, 16]",448,Dense,"[None, 10, 10, 64]",256.000000
1,Conv2D,"[None, 10, 10, 16]",2320,Dense,"[None, 10, 10, 64]",4160.000000
2,MaxPooling2D,"[None, 5, 5, 16]",0,Dense,"[None, 10, 10, 64]",4160.000000
3,Conv2D,"[None, 5, 5, 16]",2320,Flatten,"[None, 6400]",0.000000
4,Conv2D,"[None, 5, 5, 16]",2320,Dense,"[None, 3]",19203.000000
5,MaxPooling2D,"[None, 2, 2, 16]",0,None,None,nan
6,Dropout,"[None, 2, 2, 16]",0,None,None,nan
7,Flatten,"[None, 64]",0,None,None,nan
8,Dense,"[None, 50]",3250,None,None,nan
9,Dropout,"[None, 50]",0,None,None,nan


## Optional: Tune model within range of hyperparameters with Keras Tuner

*Simple example shown below. Consult [documentation](https://keras.io/guides/keras_tuner/getting_started/) to see full functionality.*

In [ ]:
! pip install keras_tuner

In [19]:
#Separate validation data 
from sklearn.model_selection import train_test_split
x_train_split, x_val, y_train_split, y_val = train_test_split(
     X_train, y_train, test_size=0.2, random_state=42)

In [20]:
import keras_tuner as kt
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers

#Define model structure & parameter search space with function
def build_model(hp):
    with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
                                 
        model = Sequential()
        model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=[10, 10, 3]))
        model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
        model.add(MaxPooling2D(pool_size=2)) 
        model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
        model.add(Conv2D(filters=hp.Int("filters", min_value=8, max_value=32, step=8), #range 8-32 inclusive, minimum step between tested values is 8
                         kernel_size=3, padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(0.3))
        model.add(Flatten())
        model.add(Dense(50, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(3, activation='softmax'))

        # Compile model...
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

        return model 

#initialize the tuner (which will search through parameters)
tuner = kt.RandomSearch(
    hypermodel=build_model, 
    objective="val_accuracy", # objective to optimize
    max_trials=2, #max number of trials to run during search
    executions_per_trial=1, #higher number reduces variance of results; guages model performance more accurately 
    overwrite=True,
    directory="tuning_model",
    project_name="tuning_units",
)

tuner.search(x_train_split, y_train_split, epochs=1, validation_data=(x_val, y_val))


Trial 2 Complete [00h 00m 09s]
val_accuracy: 0.492664098739624

Best val_accuracy So Far: 0.5104247331619263
Total elapsed time: 00h 00m 16s


In [21]:
# Build model with best hyperparameters

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
tuned_model = build_model(best_hps[0])
# Fit with the entire dataset.
tuned_model.fit(x=X_train, y=y_train, epochs=1)


203/203 [==============================] - 6s 19ms/step - loss: 1.1556 - accuracy: 0.4467


In [22]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(tuned_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("tuned_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [23]:
#Submit Model 3: 

#-- Generate predicted y values (Model 3)
prediction_column_index=tuned_model.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 3 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "tuned_model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

286/286 [==============================] - 3s 8ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 3

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2637


In [24]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dropout_layers,flatten_layers,maxpooling2d_layers,conv2d_layers,dense_layers,relu_act,softmax_act,loss,optimizer,memory_size,username,version
0,52.63%,52.76%,55.94%,58.26%,keras,False,True,Sequential,11,10811,2.000000,1,2.000000,4.000000,2,5,1,str,RMSprop,44936,COMS_NNDL,1
1,44.04%,44.07%,54.71%,51.22%,keras,False,True,Sequential,5,27779,nan,1,nan,nan,4,3,1,str,RMSprop,112168,COMS_NNDL,2
2,38.43%,24.49%,44.12%,36.77%,keras,False,True,Sequential,11,10811,2.000000,1,2.000000,4.000000,2,5,1,str,RMSprop,44936,COMS_NNDL,3


In [25]:
# Compare two or more models 
data=mycompetition.compare_models([1, 2, 3], verbose=1)
mycompetition.stylize_compare(data)

,Model_1_Layer,Model_1_Shape,Model_1_Params,Model_2_Layer,Model_2_Shape,Model_2_Params,Model_3_Layer,Model_3_Shape,Model_3_Params
0,Conv2D,"[None, 10, 10, 16]",448,Dense,"[None, 10, 10, 64]",256.000000,Conv2D,"[None, 10, 10, 16]",448
1,Conv2D,"[None, 10, 10, 16]",2320,Dense,"[None, 10, 10, 64]",4160.000000,Conv2D,"[None, 10, 10, 16]",2320
2,MaxPooling2D,"[None, 5, 5, 16]",0,Dense,"[None, 10, 10, 64]",4160.000000,MaxPooling2D,"[None, 5, 5, 16]",0
3,Conv2D,"[None, 5, 5, 16]",2320,Flatten,"[None, 6400]",0.000000,Conv2D,"[None, 5, 5, 16]",2320
4,Conv2D,"[None, 5, 5, 16]",2320,Dense,"[None, 3]",19203.000000,Conv2D,"[None, 5, 5, 16]",2320
5,MaxPooling2D,"[None, 2, 2, 16]",0,None,None,nan,MaxPooling2D,"[None, 2, 2, 16]",0
6,Dropout,"[None, 2, 2, 16]",0,None,None,nan,Dropout,"[None, 2, 2, 16]",0
7,Flatten,"[None, 64]",0,None,None,nan,Flatten,"[None, 64]",0
8,Dense,"[None, 50]",3250,None,None,nan,Dense,"[None, 50]",3250
9,Dropout,"[None, 50]",0,None,None,nan,Dropout,"[None, 50]",0
